In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import cv2
import tensorflow as tf
from tensorflow import keras

In [ ]:

DATASET_PATH = '/content/drive/My Drive/University Of Stirling/Dissertation/retinal-rec/Datasets/APTOS2019'
TRAIN_PATH = DATASET_PATH + "/train_images/"
TRAIN_PREP_PATH = DATASET_PATH + "/train_preprocessed/"
CLASSES = [ "No DR", "Mild", "Moderate", "Severe", "Proliferative DR" ]
N_CLASSES = 5
IMG_SIZE = (512, 512)

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

os.chdir(DATASET_PATH)
print("CWD:",os.getcwd())

Mounted at /content/drive
CWD: /content/drive/My Drive/University Of Stirling/Dissertation/retinal-rec/Datasets/APTOS2019


In preparation for the NN, it is necessary to create the correct directory structure.

In [ ]:
if not os.path.exists(DATASET_PATH + "/train"):
  os.mkdir(DATASET_PATH + "/train")
  os.mkdir(DATASET_PATH + "/train/0")
  os.mkdir(DATASET_PATH + "/train/1")
  os.mkdir(DATASET_PATH + "/train/2")
  os.mkdir(DATASET_PATH + "/train/3")
  os.mkdir(DATASET_PATH + "/train/4")

if not os.path.exists(DATASET_PATH + "/validation"):
  os.mkdir(DATASET_PATH + "/validation")
  os.mkdir(DATASET_PATH + "/validation/0")
  os.mkdir(DATASET_PATH + "/validation/1")
  os.mkdir(DATASET_PATH + "/validation/2")
  os.mkdir(DATASET_PATH + "/validation/3")
  os.mkdir(DATASET_PATH + "/validation/4")

if not os.path.exists(DATASET_PATH + "/test"):
  os.mkdir(DATASET_PATH + "/test")
  os.mkdir(DATASET_PATH + "/test/0")
  os.mkdir(DATASET_PATH + "/test/1")
  os.mkdir(DATASET_PATH + "/test/2")
  os.mkdir(DATASET_PATH + "/test/3")
  os.mkdir(DATASET_PATH + "/test/4")


In [ ]:
from sklearn.model_selection import train_test_split
import shutil

def read_dataset():
  df = pd.read_csv(DATASET_PATH + "/train.csv")
  
  # Train-test split
  train, test = train_test_split(df, test_size=.2)
  train, valid = train_test_split(train, test_size=.3)

  for image in train.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/train/" + str(image[1]) + "/" + image[0] + ".png")

  for image in valid.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/validation/" + str(image[1]) + "/" + image[0] + ".png")

  for image in test.values:
    shutil.copyfile(TRAIN_PREP_PATH + image[0] + ".png", DATASET_PATH + "/test/" + str(image[1]) + "/" + image[0] + ".png")
  
  return (train, valid, test)


train, valid, test = read_dataset()

print(train.shape, valid.shape, test.shape)


(2050, 2) (879, 2) (733, 2)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

df = pd.read_csv(DATASET_PATH + "/train.csv")
  
# Train-test split
#train, test = train_test_split(df, test_size=.2)

# create a data generator
datagen = ImageDataGenerator()

# load and iterate training dataset
train_it = datagen.flow_from_directory(DATASET_PATH + "/train/", class_mode='categorical', batch_size=32)
valid_it = datagen.flow_from_directory(DATASET_PATH + "/validation/", class_mode='categorical', batch_size=32)

model=keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3)),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2)), 
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(5,activation='softmax')  
])

#model.build((256,512,512,3))
model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), metrics=["accuracy"], run_eagerly=True)


model.fit(train_it, validation_data=valid_it, epochs=10, verbose=1) 

Found 2050 images belonging to 5 classes.
Found 879 images belonging to 5 classes.
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_66 (Conv2D)          (None, 62, 62, 128)       46592     
                                                                 
 batch_normalization_65 (Bat  (None, 62, 62, 128)      512       
 chNormalization)                                                
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 31, 31, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_67 (Conv2D)          (None, 31, 31, 256)       819456    
                                                                 
 batch_normalization_66 (Bat  (None, 31, 31, 256)      1024      
 chNormalization)                   

In [ ]:
test_it = datagen.flow_from_directory(DATASET_PATH + "/test/", class_mode='categorical', batch_size=32, shuffle = False)

predictions = model.predict(test_it)
loss, acc = model.evaluate(test_it, verbose=1)
print(loss,acc)

Found 733 images belonging to 5 classes.
1260
23/23 [==============================] - 34s 1s/step - loss: 2.0655 - accuracy: 0.3602
2.065523386001587 0.36016371846199036


## PyTorch

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torch.utils.data import DataLoader 
import random
from torchvision import models
import torchvision.transforms as transforms

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path + ".png")
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize(224),
    transforms.ToTensor()
    ])

dataset = CustomImageDataset(DATASET_PATH + "/train.csv", TRAIN_PREP_PATH, transform = train_transforms)

train_size = int(0.8 * len(dataset))
test_size = (len(dataset) - train_size)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_size = int(0.8 * train_size)
valid_size = (len(dataset) - train_size - test_size)

train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
model = models.alexnet(pretrained=False, num_classes=5)
model = model.to(device)
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
import time

num_epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Ensures network is in train mode
model.train()

#
#for epoch in range(num_epochs):
#  for data, targets in iter(train_loader):
    # Sends data and targets to device
#    data = data.to(device)
#    targets = targets.to(device)

    # Acquires the network's best guesses at each class
#    results = model(data)

    # Computes loss
#    loss = loss_fn(results, targets)

    # Updates model
#    optimizer.zero_grad()
#    loss.backward()
#    optimizer.step()

start_time = time.time()
for epoch in range(num_epochs): 
    loss_ep = 0.0
    
    for data, targets in iter(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        # Acquires the network's best guesses at each class
        results = model(data)

        ## Forward Pass
        optimizer.zero_grad()
        loss = loss_fn(results,targets)
        loss.backward()
        optimizer.step()
        loss_ep += loss.item()

    print(f"Loss in epoch {epoch} :::: {loss_ep/len(train_loader)}")

    with torch.no_grad():
        num_correct = 0
        num_samples = 0
        for data, targets in iter(valid_loader):
            data = data.to(device=device)
            targets = targets.to(device=device)

            # Acquires the network's best guesses at each class
            results = model(data)

            ## Forward Pass
            _, predictions = results.max(1)
            num_correct += (predictions == targets).sum()
            num_samples += predictions.size(0)
        print(
            f"Validation: Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}"
        )



elapsed_time = time.time() - start_time
print ("Spent ", elapsed_time, " seconds training for ", num_epochs, " epoch(s) on a single core.")

Loss in epoch 0 :::: 1.295005300549517
Got 282 / 586 with accuracy 48.12
Loss in epoch 1 :::: 1.2929341991403405
Got 282 / 586 with accuracy 48.12
Loss in epoch 2 :::: 1.294068764195914
Got 282 / 586 with accuracy 48.12
Loss in epoch 3 :::: 1.2893137472583165
Got 282 / 586 with accuracy 48.12


KeyboardInterrupt: ignored

In [ ]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 48 %
